In [1]:
import os
import keras
import random
import pathlib
import numpy as np
import tensorflow as tf

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave

Using TensorFlow backend.


In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7910459749099928503
]


In [4]:
'''
Second experiment testing on 200 images
'''
imgs = []
for filename in os.listdir('../../images/Train/'):
    imgs.append(img_to_array(load_img('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/images/Train/'+filename)))
imgs = np.array(imgs, dtype=float)
''' 
Convert RBG 255 colors into base 2 binary for computational ease (binary)
RGB coefficients from 0-255 would be too high for our model to process (given a typical learning rate),
thus convert to range 0-1.0
'''
imgs = 1.0/255*imgs

MemoryError: 

In [3]:
# Image transformer
image_gen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=10,
        horizontal_flip=True)

batch_size = 16

# function to split training set X train, y train and produce augmented images
def image_a_b_gen(batch_size):
    for i in image_gen.flow(imgs, batch_size=batch_size):
        lab = rgb2lab(i)
        X_train = lab[:,:,:,0]
        y_train = lab[:,:,:,1:] / 128
        yield (X_train.reshape(X_train.shape+(1,)), y_train)

In [3]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

# Finish model
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
# Train model
model.fit_generator(image_a_b_gen(batch_size), steps_per_epoch=4000, epochs=1)

In [7]:
model.save('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/2nd_expt_3.h5')

In [4]:
model.load_weights('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/Fails/Experiment_2/2nd_expt_3.h5')

W0716 14:26:44.280590  1152 deprecation_wrapper.py:119] From C:\Users\n3rDx\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0716 14:26:44.281591  1152 deprecation_wrapper.py:119] From C:\Users\n3rDx\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [6]:
# Load black and white images
test = []
for filename in os.listdir('../Experiment_3/test'):
        test.append(img_to_array(load_img('C:/Users/n3rDx/Desktop/Homework Upload/Capstone/Fails/Experiment_3/test/'+filename)))
test = np.array(test, dtype=float)
test = rgb2lab(test/255.0)[:,:,:,0]
test = test.reshape(test.shape+(1,))

In [7]:
# Test model
output = model.predict(test)
output = output * 128


In [8]:
# Output colorizations
for i in range(len(output)):
        cur = np.zeros((256, 256, 3))
        cur[:,:,0] = test[i][:,:,0]
        cur[:,:,1:] = output[i]
        imsave("C:/Users/n3rDx/Desktop/Homework Upload/Capstone/Fails/Experiment_2/"+str(i)+".jpg", lab2rgb(cur))

W0716 14:28:56.472811  1152 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0716 14:28:56.521847  1152 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0716 14:28:56.552869  1152 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0716 14:28:56.597900  1152 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0716 14:28:56.643933  1152 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0716 14:28:56.695969  1152 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0716 14:28:56.745005  1152 util.py:64] Lossy 

In [3]:
# Train test split
train = imgs[:195]
test = imgs[195:]

In [4]:
# Import map images into the lightness - a/b colorspace
X_test = rgb2lab(test)[:,:,:,0]
y_test = rgb2lab(test)[:,:,:,1:]/128
X_train = rgb2lab(train)[:,:,:,0]
y_train = rgb2lab(train)[:,:,:,1:]/128

In [5]:
#Reshape for input into CNN
X_train = X_train.reshape(X_train.shape+(1,))
X_test = X_test.reshape(X_test.shape+(1,))

In [ ]:
#Train the neural network
model.fit(X_train, y_train, batch_size=batch_size, epochs=50)